# Building event features

> These are Go notebooks: In order to use the GoNB Jupyter Kernel, please install GoNB from here: https://github.com/janpfeifer/gonb

Note also that for local package development, you can put: `!*go mod edit -replace "github.com/umbralcalc/trywizard=/path/to/trywizard"` at the top of any cell.

In this notebook we're building event features from local csv files which aggregate information over all the data sources listed in the `README.md`.

In [ ]:
import (
	"os"
	"github.com/go-gota/gota/dataframe"
	"github.com/go-echarts/go-echarts/v2/charts"
	"github.com/go-echarts/go-echarts/v2/opts"
	gonb_echarts "github.com/janpfeifer/gonb-echarts"
)

%%

// Retrieving from csv
file, _ := os.Open("../dat/events.csv")
df := dataframe.ReadCSV(file)

// Grouping event counts by time
timeStrs := df.Col("time").Records()
timeInts := make([]int, len(timeStrs))
for i, t := range timeStrs {
	val, err := strconv.Atoi(t[0:len(t)-1])
	if err != nil {
		panic(err)
	}
	timeInts[i] = val
}
df = df.Mutate(series.New(series.Ints(timeInts), series.Int, "time_ints"))
gdf := df.GroupBy("time_ints").Aggregation(
	[]dataframe.AggregationType{
		dataframe.Aggregation_COUNT,
	},
	[]string{"event_type"},
)
gdf = gdf.Arrange(
    dataframe.Sort("time_ints"),
)
fmt.Println(gdf)

// Plotting the event counts by time
scatter := charts.NewScatter()
scatter.SetGlobalOptions(
	charts.WithXAxisOpts(opts.XAxis{
		Name: "time_ints",
	}),
	charts.WithTooltipOpts(opts.Tooltip{
		Trigger:   "item",
		Formatter: "({c})",
	}),
)
xValues, _ := gdf.Col("time_ints").Int()
plotData := make([]opts.ScatterData, 0)
for j, yYalue := range gdf.Col("event_type_COUNT").Float() {
	plotData = append(plotData, opts.ScatterData{
		Value: []interface{}{xValues[j], yYalue},
	})
}
scatter.AddSeries("event counts", plotData)
gonb_echarts.Display(scatter, "width: 1024px; height:400px; background: white;")